<a href="https://colab.research.google.com/github/trduong01/TranNguyenDuong_2001230135/blob/main/Ch%C3%A0o_m%E1%BB%ABng_b%E1%BA%A1n_%C4%91%E1%BA%BFn_v%E1%BB%9Bi_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ======================= BÀI 1: Đọc file + tô màu =======================

# --- Tạo file adjacency.txt tự động trong Colab ---
matrix_text = """0 1 1 0 1 0
1 0 1 1 0 1
1 1 0 1 1 0
0 1 1 0 0 1
1 0 1 0 0 1
0 1 0 1 1 0
"""

with open("adjacency.txt", "w") as f:
    f.write(matrix_text)

print("Đã tạo file adjacency.txt")


# --- Hàm đọc ma trận kề ---
def read_adj_matrix(filename):
    G = []
    with open(filename, "r") as f:
        for line in f:
            row = list(map(int, line.split()))
            G.append(row)
    return G


# --- Hàm tô màu đồ thị ---
def graph_coloring(G):
    n = len(G)
    nodes = [chr(i + 65) for i in range(n)]  # A, B, C,...

    degree = [sum(G[i]) for i in range(n)]
    colorDict = {nodes[i]: ["Blue", "Red", "Yellow", "Green"] for i in range(n)}

    # sắp xếp theo bậc giảm dần
    sorted_nodes = sorted(nodes, key=lambda x: degree[nodes.index(x)], reverse=True)

    idx = {nodes[i]: i for i in range(n)}
    result = {}

    for node in sorted_nodes:
        avail = colorDict[node][0]
        result[node] = avail

        for j in range(n):
            if G[idx[node]][j] == 1 and avail in colorDict[nodes[j]]:
                colorDict[nodes[j]].remove(avail)

    return result


# ================== CHẠY ==================
G = read_adj_matrix("adjacency.txt")
solution = graph_coloring(G)

print("KẾT QUẢ TÔ MÀU:")
for t in sorted(solution):
    print(f"Đỉnh {t} = {solution[t]}")


Đã tạo file adjacency.txt
KẾT QUẢ TÔ MÀU:
Đỉnh A = Yellow
Đỉnh B = Blue
Đỉnh C = Red
Đỉnh D = Yellow
Đỉnh E = Blue
Đỉnh F = Red


In [5]:
# ======================= BÀI 2: Chương trình con =======================

def calc_degree(G):
    return [sum(row) for row in G]

def init_colors(nodes):
    return {n: ["Blue", "Red", "Yellow", "Green"] for n in nodes}

def sort_nodes_by_degree(nodes, degree):
    return sorted(nodes, key=lambda x: degree[nodes.index(x)], reverse=True)

def color_graph(G):
    n = len(G)
    nodes = [chr(65+i) for i in range(n)]

    deg = calc_degree(G)
    sorted_nodes = sort_nodes_by_degree(nodes, deg)
    colorDict = init_colors(nodes)
    index = {nodes[i]: i for i in range(n)}
    result = {}

    for node in sorted_nodes:
        color = colorDict[node][0]
        result[node] = color

        for j in range(n):
            if G[index[node]][j] == 1 and color in colorDict[nodes[j]]:
                colorDict[nodes[j]].remove(color)

    return result

# chạy thử
G = read_adj_matrix("adjacency.txt")
print(color_graph(G))


{'B': 'Blue', 'C': 'Red', 'A': 'Yellow', 'D': 'Yellow', 'E': 'Blue', 'F': 'Red'}


In [ ]:
# ======================= BÀI 3: TSP =======================
from itertools import permutations

def tsp(cost):
    n = len(cost)
    cities = range(n)
    best_path = None
    best_cost = float("inf")

    for perm in permutations(cities):
        s = 0
        for i in range(n - 1):
            s += cost[perm[i]][perm[i + 1]]
        s += cost[perm[-1]][perm[0]]  # quay lại điểm đầu

        if s < best_cost:
            best_cost = s
            best_path = perm

    return best_path, best_cost

# ví dụ
cost_matrix = [
    [0, 2, 9, 10],
    [1, 0, 6, 4],
    [15, 7, 0, 8],
    [6, 3, 12, 0]
]

path, cost = tsp(cost_matrix)
print("Chu trình tối ưu:", path)
print("Chi phí tối thiểu:", cost)


In [6]:
# =================== BÀI TẬP VỀ NHÀ: GIẢI THUẬT DI TRUYỀN ===================

import random

# ====== DỮ LIỆU ======
teachers = ["T1", "T2", "T3"]            # Giáo viên
subjects = ["Math", "IT"]               # Môn học
slots = 25                               # 5 ngày × 5 tiết

# ====== Tạo 1 thời khóa biểu ngẫu nhiên ======
def random_schedule():
    return [(random.choice(teachers), random.choice(subjects)) for _ in range(slots)]

# ====== HÀM FITNESS ======
def fitness(schedule):
    penalty = 0

    # Ràng buộc 1: Giáo viên không được dạy 2 tiết cùng lúc (theo từng ngày)
    for day in range(5):
        day_slots = schedule[day*5 : day*5 + 5]  # 5 tiết mỗi ngày
        seen = []
        for (gv, mh) in day_slots:
            if gv in seen:
                penalty += 5   # phạt vì trùng tiết
            seen.append(gv)

    # Ràng buộc 2: Giáo viên không dạy quá lệch (cân bằng số tiết)
    count_teacher = {t: 0 for t in teachers}
    for gv, mh in schedule:
        count_teacher[gv] += 1

    avg = slots / len(teachers)
    for t in teachers:
        penalty += abs(count_teacher[t] - avg) * 0.3

    return -penalty   # fitness càng cao càng tốt

# ====== LAI GHÉP (crossover) ======
def crossover(p1, p2):
    cut = random.randint(1, slots-2)
    return p1[:cut] + p2[cut:]

# ====== ĐỘT BIẾN (mutation) ======
def mutate(schedule):
    idx = random.randint(0, slots-1)
    schedule[idx] = (random.choice(teachers), random.choice(subjects))


# ===================== GIẢI THUẬT DI TRUYỀN (GA) =====================
POP = 60       # số lượng cá thể
GEN = 250      # số thế hệ

population = [random_schedule() for _ in range(POP)]

for gen in range(GEN):
    # xếp theo fitness
    population.sort(key=fitness, reverse=True)
    new_pop = population[:10]  # elitism: giữ 10 cá thể tốt nhất

    # tạo thế hệ mới
    while len(new_pop) < POP:
        p1, p2 = random.sample(population[:20], 2)
        child = crossover(p1, p2)
        if random.random() < 0.1:
            mutate(child)
        new_pop.append(child)

    population = new_pop

# ===================== KẾT QUẢ TỐT NHẤT =====================
best = population[0]

print("===== THỜI KHÓA BIỂU TỐT NHẤT TÌM ĐƯỢC =====")
for i in range(slots):
    day = i // 5 + 1
    period = i % 5 + 1
    gv, mh = best[i]
    print(f"Ngày {day}, Tiết {period}: {gv} – {mh}")


===== THỜI KHÓA BIỂU TỐT NHẤT TÌM ĐƯỢC =====
Ngày 1, Tiết 1: T2 – IT
Ngày 1, Tiết 2: T1 – Math
Ngày 1, Tiết 3: T3 – Math
Ngày 1, Tiết 4: T1 – IT
Ngày 1, Tiết 5: T1 – Math
Ngày 2, Tiết 1: T2 – IT
Ngày 2, Tiết 2: T1 – Math
Ngày 2, Tiết 3: T1 – Math
Ngày 2, Tiết 4: T3 – IT
Ngày 2, Tiết 5: T3 – IT
Ngày 3, Tiết 1: T1 – IT
Ngày 3, Tiết 2: T2 – IT
Ngày 3, Tiết 3: T3 – Math
Ngày 3, Tiết 4: T3 – Math
Ngày 3, Tiết 5: T3 – Math
Ngày 4, Tiết 1: T1 – IT
Ngày 4, Tiết 2: T2 – Math
Ngày 4, Tiết 3: T2 – Math
Ngày 4, Tiết 4: T2 – Math
Ngày 4, Tiết 5: T3 – IT
Ngày 5, Tiết 1: T3 – Math
Ngày 5, Tiết 2: T1 – IT
Ngày 5, Tiết 3: T2 – IT
Ngày 5, Tiết 4: T2 – IT
Ngày 5, Tiết 5: T1 – IT
